In [5]:
!pip install openpyxl

In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
import plotly.graph_objects as go
from scipy.interpolate import griddata

In [12]:
warnings.simplefilter("ignore")
df_coffee = pd.read_excel('/content/coffee chain data (1).xlsx', engine='openpyxl')


In [13]:
df_coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4248 entries, 0 to 4247
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Area Code       4248 non-null   int64         
 1   Date            4248 non-null   datetime64[ns]
 2   Market Size     4248 non-null   object        
 3   Market          4248 non-null   object        
 4   Product Line    4248 non-null   object        
 5   Product Type    4248 non-null   object        
 6   Product         4248 non-null   object        
 7   State           4248 non-null   object        
 8   Type            4248 non-null   object        
 9   Budget COGS     4248 non-null   int64         
 10  Budget Margin   4248 non-null   int64         
 11  Budget Profit   4248 non-null   int64         
 12  Budget Sales    4248 non-null   int64         
 13  COGS            4248 non-null   int64         
 14  Inventory       4248 non-null   int64         
 15  Marg

In [48]:
df_coffee['Date'] = pd.to_datetime(df_coffee['Date'])

sales_trend = df_coffee.groupby(['Date', 'Product Type'])['Sales'].sum().reset_index()

fig = px.line(sales_trend, x='Date', y='Sales', color='Product Type',
              title='Sales Trend Over Time by Product Type',
              labels={'Date': 'Date', 'Sales': 'Total Sales', 'Product Type': 'Product Type'},
              markers=True)

fig.show()

In [49]:
fig = px.box(df_coffee, x='Market Size', y='Sales',
              title='Sales Distribution by Market Size',
              labels={'Market Size': 'Market Size', 'Sales': 'Sales'},
              color='Market Size')
fig.show()

In [17]:
fig = px.scatter(df_coffee, x='Sales', y='Profit',
                  title='Sales vs. Profit',
                  labels={'Sales': 'Sales', 'Profit': 'Profit'},
                  color='Product Type', opacity=0.7)
fig.show()

In [52]:
state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

df_coffee['State_Abbr'] = df_coffee['State'].map(state_abbreviations)

sales_by_state = df_coffee.groupby('State_Abbr')['Sales'].sum().reset_index()

fig6 = px.choropleth(sales_by_state,
                     locations='State_Abbr',
                     locationmode='USA-states',
                     color='Sales',
                     scope='usa',
                     title='Total Sales by State',
                     labels={'Sales': 'Total Sales'})

fig6.show()

In [53]:
fig = px.sunburst(df_coffee,
                  path=['State', 'Market Size', 'Product Line'],
                  values='Sales',
                  title="Sunburst Chart: Sales Breakdown by State, Market Size, and Product Line",
                  color='Sales',
                  color_continuous_scale='RdBu'
                  )
fig.show()

In [26]:
sales_coffee = df_coffee.groupby(['Date', 'Product'])['Sales'].sum().reset_index()
fig_bar_coffee = px.bar(sales_coffee, x='Product', y='Sales', color='Product',
                         animation_frame=sales_coffee['Date'].dt.strftime('%Y-%m'),
                         title='Animated Sales Over Time by Coffee Type')
fig_bar_coffee.show()

In [56]:
df_coffee['Date'] = pd.to_datetime(df_coffee['Date'])
sales_coffee = df_coffee.groupby('Product')['Sales'].sum().reset_index()
fig_polar = px.bar_polar(sales_coffee, r='Sales', theta='Product', color='Product',
                         title='Sales Contribution by Coffee Type (Polar Chart)',
                         template='plotly_dark')
fig_polar.show()

In [34]:
fig = px.scatter_3d(df, x="Sales", y="Profit", z="Margin",
                     color="Product Type", hover_data=["Product", "Market"],
                     title="3D Scatter Plot of Sales, Profit, and Margin")

fig.show()


In [54]:
df_coffee = df_coffee.dropna(subset=["Sales", "Profit", "Inventory"])

# Extract necessary columns
x = df_coffee["Sales"].values   # X-axis: Product Sales
y = df_coffee["Profit"].values  # Y-axis: Profit
z = df_coffee["Inventory"].values  # Z-axis: Inventory (or use 'Revenue' if available)

# Define grid size
grid_x, grid_y = np.mgrid[x.min():x.max():30j, y.min():y.max():30j]

# Interpolate inventory values using griddata
grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

# Create a 3D surface plot
fig_surface = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y)])

fig_surface.update_layout(
    title="3D Surface Plot of Product Sales, Profit, and Inventory",
    scene=dict(
        xaxis_title="Product Sales",
        yaxis_title="Profit",
        zaxis_title="Inventory"
    )
)

# Show the plot
fig_surface.show()

# Save the plot as an interactive HTML file
fig_surface.write_html("3d_surface_product_sales_profit.html")